<a href="https://colab.research.google.com/github/Phil-Schwarz/TQA/blob/squbr%2Fcombiner_network/combiner_network/LSTM_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  Get ARC Dataset from my GoogleDrive

In [ ]:
##  Get ARC Dataset from my GoogleDrive
!gdown --id 1Fp5TIgoqJLg0uL69WSWlaztN7EclxwJl 

Downloading...
From: https://drive.google.com/uc?id=1Fp5TIgoqJLg0uL69WSWlaztN7EclxwJl
To: /content/ARC-V1-Feb2018.zip
681MB [00:04, 142MB/s]


## Extract Dataset


In [ ]:
!jar xvf ARC-V1-Feb2018.zip

  created: ARC-V1-Feb2018-2/
 inflated: ARC-V1-Feb2018-2/.DS_Store
  created: __MACOSX/
  created: __MACOSX/ARC-V1-Feb2018-2/
 inflated: __MACOSX/ARC-V1-Feb2018-2/._.DS_Store
  created: ARC-V1-Feb2018-2/ARC-Challenge/
 inflated: ARC-V1-Feb2018-2/ARC-Challenge/ARC-Challenge-Dev.csv
  created: __MACOSX/ARC-V1-Feb2018-2/ARC-Challenge/
 inflated: __MACOSX/ARC-V1-Feb2018-2/ARC-Challenge/._ARC-Challenge-Dev.csv
 inflated: ARC-V1-Feb2018-2/ARC-Challenge/ARC-Challenge-Dev.jsonl
 inflated: __MACOSX/ARC-V1-Feb2018-2/ARC-Challenge/._ARC-Challenge-Dev.jsonl
 inflated: ARC-V1-Feb2018-2/ARC-Challenge/ARC-Challenge-Test.csv
 inflated: __MACOSX/ARC-V1-Feb2018-2/ARC-Challenge/._ARC-Challenge-Test.csv
 inflated: ARC-V1-Feb2018-2/ARC-Challenge/ARC-Challenge-Test.jsonl
 inflated: __MACOSX/ARC-V1-Feb2018-2/ARC-Challenge/._ARC-Challenge-Test.jsonl
 inflated: ARC-V1-Feb2018-2/ARC-Challenge/ARC-Challenge-Train.csv
 inflated: __MACOSX/ARC-V1-Feb2018-2/ARC-Challenge/._ARC-Challenge-Train.csv
 inflated: ARC-V1-F

## Prepare Dataset for training

In [ ]:
!pip install jsonlines

In [ ]:
import jsonlines
import pandas as pd

In [ ]:
path_easy = "ARC-V1-Feb2018-2/ARC-Easy/ARC-Easy-Train.jsonl"
path_chall = "ARC-V1-Feb2018-2/ARC-Challenge/ARC-Challenge-Train.jsonl"

def extract_questions_and_answers(path):
  data_rows = []
  answer_choices = []

  with jsonlines.open(path) as reader:
    for obj in reader:
      id = obj["id"]
      question = obj["question"]["stem"]
      answer_key = obj["answerKey"]# more possible answers
      for answer_choice in obj["question"]["choices"]:
        answer_choices.append(answer_choice["label"]+": "+answer_choice["text"])

      data_rows.append({
            "id":id,
            "question":question,
            "answer_choices":answer_choices,
            "answer_key":answer_key
      })
  return pd.DataFrame(data_rows)

extract_questions_and_answers(path_easy)
# BASE_DIR = os.path.abspath(os.path.dirname(__file__))
# MODELS_DIR = os.path.abspath(os.path.join(BASE_DIR, "models"))
# DATA_DIR = os.path.abspath(os.path.join(BASE_DIR, "data"))

# EASY_TRAIN_DATA_PATH = os.path.join(DATA_DIR, "easy_train.json")
# EASY_VAL_DATA_PATH = os.path.join(DATA_DIR, "easy_validation.json")
# EASY_TEST_DATA_PATH = os.path.join(DATA_DIR, "easy_test.json")

# CHALLENGE_TRAIN_DATA_PATH = os.path.join(DATA_DIR, "challenge_train.json")
# CHALLENGE_VAL_DATA_PATH = os.path.join(DATA_DIR, "challenge_validation.json")
# CHALLENGE_TEST_DATA_PATH = os.path.join(DATA_DIR, "challenge_test.json")

# TRAIN_DATA_PATH = EASY_TRAIN_DATA_PATH
# VAL_DATA_PATH = EASY_VAL_DATA_PATH
# TEST_DATA_PATH = EASY_TEST_DATA_PATH

,id,question,answer_choices,answer_key
0,Mercury_7220990,Which factor will most likely cause a person t...,"[A: a leg muscle relaxing after exercise, B: a...",B
1,MCAS_2007_8_5189,Lichens are symbiotic organisms made of green ...,"[A: a leg muscle relaxing after exercise, B: a...",B
2,Mercury_SC_401169,When a switch is used in an electrical circuit...,"[A: a leg muscle relaxing after exercise, B: a...",D
3,MCAS_2004_8_27,Which of the following is an example of an ass...,"[A: a leg muscle relaxing after exercise, B: a...",A
4,NYSEDREGENTS_2006_8_10,"Rocks are classified as igneous, metamorphic, ...","[A: a leg muscle relaxing after exercise, B: a...",3
...,...,...,...,...
2246,CSZ_2009_8_CSZ20770,"Iron oxides, such as rust, form when iron meta...","[A: a leg muscle relaxing after exercise, B: a...",C
2247,Mercury_7181318,When water evaporates from Earth's surface int...,"[A: a leg muscle relaxing after exercise, B: a...",C
2248,Mercury_7010080,Which process directly adds carbon into the at...,"[A: a leg muscle relaxing after exercise, B: a...",C
2249,NCEOGA_2013_8_37,Scientists think that dolphins and whales may ...,"[A: a leg muscle relaxing after exercise, B: a...",D


In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
# input_size = 784 # 28x28
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

input_size = 3 # one row at a time (tf-idf, destilbert, albert)                 #28
sequence_length = 20 # amount of documents                                      #28
hidden_size = 100                                                               #128
num_layers = 2 # amount of rnn/lstm boxes

# MNIST dataset 
# train_dataset = torchvision.datasets.MNIST(root='./data', 
#                                            train=True, 
#                                            transform=transforms.ToTensor(),  
#                                            download=True)

# test_dataset = torchvision.datasets.MNIST(root='./data', 
#                                           train=False, 
#                                           transform=transforms.ToTensor())




# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)


# Fully connected neural network with one hidden layer
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # -> x needs to be: (batch_size, seq, input_size)                         #what is batchsize -Batch size is a term used in machine learning
                                                                                  # and refers to the number of training examples utilized in one iteration. 
        
        # or:
        #self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        #self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        
        # x: (n, 28, 28), h0: (2, n, 128)
        
        # Forward propagate RNN
        #out, _ = self.rnn(x, h0)  
        # or:
        out, _ = self.lstm(x, (h0,c0))                                            # underscore because we dont need this
        
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        # out: (n, 28, 128)
        
        # Decode the hidden state of the last time step
        out = out[:, -1, :]
        # out: (n, 128)
         
        out = self.fc(out)                                                        # fully connected
        # out: (n, 10)
        return out

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Train the model
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # origin shape: [N, 1, 28, 28]
        # resized: [N, 28, 28]

        # bring input data to apropiate format
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        # bring input data to apropiate format
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 test images: {acc} %')

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Processing...


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


RuntimeError: ignored